# Weather Prediction using RNN

In [ ]:

# Install necessary libraries
!pip install pandas numpy scikit-learn tensorflow


In [ ]:

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Load CSV file (Replace with your actual file path)
df = pd.read_csv("stations.csv")

# Display first few rows
df.head()


In [ ]:

# Define the targeted station ID (output)
target_station_id = 101  # Replace with your target station ID

# Define selected stations (input)
selected_station_ids = [102, 103, 104]  # Replace with your selected station IDs

# Filter dataset for input and target stations
target_df = df[df["Station ID"] == target_station_id].copy()
input_df = df[df["Station ID"].isin(selected_station_ids)].copy()

# Sort data by date
target_df = target_df.sort_values(by="Date")
input_df = input_df.sort_values(by="Date")

# Normalize features
scaler = MinMaxScaler()
target_df.iloc[:, 2:] = scaler.fit_transform(target_df.iloc[:, 2:])
input_df.iloc[:, 2:] = scaler.transform(input_df.iloc[:, 2:])

# Convert to NumPy arrays
X = input_df.iloc[:, 2:].values  # Input features
Y = target_df.iloc[:, 2:].values  # Output features


In [ ]:

# Function to create sequences
def create_sequences(X, Y, time_steps=7):
    X_seq, Y_seq = [], []
    for i in range(len(X) - time_steps):
        X_seq.append(X[i : i + time_steps])
        Y_seq.append(Y[i + time_steps])  # Predicting next day after sequence
    return np.array(X_seq), np.array(Y_seq)

time_steps = 7  # Predicting next 7 days
X_train, Y_train = create_sequences(X, Y, time_steps)


In [ ]:

# Define RNN model
model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(time_steps, X_train.shape[2])),
    LSTM(50, activation='relu'),
    Dense(Y_train.shape[1])  # Output layer for multi-feature prediction
])

# Compile model
model.compile(optimizer="adam", loss="mse")

# Train model (you can increase epochs for better performance)
model.fit(X_train, Y_train, epochs=50, batch_size=16, verbose=1)


In [ ]:

# Predict next 7 days
test_input = X_train[-1].reshape(1, time_steps, X_train.shape[2])  # Take the last sequence
predicted_weather = model.predict(test_input)

# Display Predictions
print("Predicted Weather for Next 7 Days:", predicted_weather)
